In [19]:
import pandas as pd
from pandas import json_normalize
import requests
import json
import csv
import os
import re
from quant_preprocess import query_and_preprocess_data
from quant_preprocess import query_and_reshape_long
from quant_preprocess import recode_long_data



#if the measurement is 750, its in ml
#if the measurement is 12, its in oz

measurements = ["oz", "ml", "lb", "l", "ounces", "g"]
numbers = ["1", "2", "3", "4", "5", "6", "7", "8", "9","0", "."]
IN_FILE_PATH = os.path.join("..//data", "items.txt")
OUTPUT_DIR = "data"
OUTPUT_PATH = os.path.join(OUTPUT_DIR, 'ingredient_prices.csv')

In [20]:
def load_ingredients():
    """Loads the list of ingredients to a list"""

    df = query_and_reshape_long()
    recoded = recode_long_data(df)
    summary = recoded[["ingredient", "amount"]].groupby("ingredient").agg(["mean", "sum"])
    return summary.index.values.tolist()

In [21]:
def read_txt_as_json():
    
    j_list = []
    
    with open(IN_FILE_PATH) as text:
        
        json_list = [line.rstrip("\n") for line in text]
        for j in json_list:
            j_list.append(json.loads(j))
        return j_list
       

In [22]:
def split(word):
    return [char for char in word]

In [23]:
 def get_product_info():
        
        j_list = read_txt_as_json()
        product_dict = {}
        for j in j_list:
            amount_temp = []
            units_temp = []
            price_temp = []
            desc_temp = []
            ingredient = j["request_parameters"]["search_term"]
            key = ingredient
            product_dict.setdefault(key, [])
            a = "search_results"
            if a not in j:
                continue
            else:
                results = j["search_results"]
                i = 0
                for r in results[0:5]:
                    n_temp = []
                    m_temp = []
                    title = r["product"]["title"]
                    chars = split(title)
                    title_list = title.split()
                    price = r["offers"]["primary"]["price"]
                    price_temp.append(price)
                    desc_temp.append(title)
                    for c in reversed(range(len(chars))):
                        if (chars[c] not in (''.join(numbers))) and (len(n_temp) > 0):
                            break
                        if c ==  0:
                            amount_temp.append("NA")
                        if chars[c] not in (''.join(numbers)):
                            continue
                        else:
                            for n in numbers:
                                if n == chars[c]:
                                    n_temp.append(chars[c])
                    amount_temp.append((''.join(n_temp))[::-1])
                    for t in reversed(range(len(title_list))):
                        for m in measurements:
                            if len(m_temp) > 0:
                                break
                            if t == 0:
                                units_temp.append("NA")
                            if m not in (title_list[t].lower()):
                                continue
                            else:
                                m_temp.append(m)
                    units_temp.append(m_temp)
                product_dict[key].append(price_temp)
                product_dict[key].append(amount_temp)
                product_dict[key].append(units_temp)
                product_dict[key].append(desc_temp)
        return product_dict


In [24]:
def drop_bad_products(product_dict):
    
    ingredients = load_ingredients()
    for i in ingredients:
        if (len(product_dict[i]) == 0):
            del product_dict[i]
        else:
            for n in range(0,3):
                if (len(product_dict[i][n]) !=5):
                    del product_dict[i]
                    break
    return product_dict
        


In [25]:
def set_index(df):
    index = ['price', 'measurement', 'units', 'description']
    df = pd.DataFrame(d)
    df['index'] = index
    df.set_index('index')
    return df

In [28]:
def to_csv(df, path):
#""Takes the dataframe and writes it to a csv""

    with open(OUT_FILE_PATH, 'w+') as out_file:
        df.to_csv(OUT_FILE_PATH)
        

In [127]:
# if __name__ == '__main__':

#     os.makedirs(OUT_DIR, exist_ok = True)
#     to_csv(item_dict, OUT_PATH)

    